In [3]:
import pandas as pd 
import numpy as np 
import tensorflow as tf

from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, Bidirectional
from modules.utils import  preprocees_order_book_data

In [4]:
X = pd.read_csv("/Users/elouan/Data/Data_ENSxCFM/X_train.csv")
y = pd.read_csv("/Users/elouan/Data/Data_ENSxCFM/y_train.csv")

In [5]:
X2 = preprocees_order_book_data(X)

/Users/elouan/Repo Github ElouanBahri/ENS_CFM_data_challenge/src/notebooks/modules/utils.py:101: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["volatility_mid_price"] = df["volatility_mid_price"].fillna(method="bfill")
/Users/elouan/anaconda3/envs/challenge_ENS_env_CFM/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [6]:
X2.head()

,column 1
0,"[[0.0, 0.0, -0.0026713808, -0.004683957, -0.05..."
1,"[[0.0, 0.0, -0.004417788, -0.004683957, -0.050..."
2,"[[0.0, 0.0, -0.004004165, -0.004683957, -0.052..."
3,"[[0.0, 0.0, -0.0030850035, -0.004683957, -0.03..."
4,"[[0.0, 1.0, -0.004004165, -0.004683957, -0.052..."


In [7]:
X3 = np.array([np.array(row) for row in X2["column 1"]])

In [21]:
X3[40][10]

array([ 7.0000000e+00,  0.0000000e+00, -4.0501230e-03, -2.7817583e-01,
       -5.3617056e-02,  7.0255384e+00,  7.8091354e+00,  0.0000000e+00,
        2.0000000e+00, -4.9999999e-03,  9.9999998e-03,  9.9999998e-03,
        0.0000000e+00, -3.7311769e-01, -3.1600000e+02,  2.5819889e-03,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  1.0000000e+00,
        0.0000000e+00,  0.0000000e+00], dtype=float32)

In [8]:
y1 = y.drop(columns=["obs_id"])
y2 = np.array(y1["eqt_code_cat"])

In [9]:
print(np.any(np.isnan(X3)))  # Check features for NaNs
print(np.any(np.isnan(y2)))

True
False


In [10]:
X3 = np.nan_to_num(X3)

In [11]:
dataset = tf.data.Dataset.from_tensor_slices((X3, y2))

In [12]:
# Calculate sizes for train and validation sets
dataset_size = len(X3)  # Total number of samples in your dataset
train_size = int(0.8 * dataset_size)

# Shuffle the dataset if needed
dataset = dataset.shuffle(buffer_size=dataset_size)

# Split the dataset
train_dataset = dataset.take(train_size)       # Take the first 80%
val_dataset = dataset.skip(train_size)         # Skip the first 80%, take the remaining 20%

# Batch both datasets if needed
batch_size = 32
train_dataset = train_dataset.batch(batch_size)
val_dataset = val_dataset.batch(batch_size)

In [39]:
tf.random.set_seed(1234) # for consistent results
model = Sequential(
    [   Bidirectional(GRU(128, return_sequences=True, input_shape = (100, 22) )),            
        Dropout(0.2),
        Bidirectional(GRU(64)),
        Dropout(0.2),
        Dense(252, activation = 'relu',   name = "L1"),
        Dense(128, activation = 'relu',   name = "L2"),
        Dense(64, activation = 'relu',   name = "L3"),
        Dense(32, activation = 'relu',   name = "L4"),
        Dense(24, activation = 'softmax',   name = "output"),

    ], name = "second_multiclass_model" 
)

/Users/elouan/anaconda3/envs/challenge_ENS_env_CFM/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [14]:
model.summary()

Model: "second_multiclass_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional_2 (Bidirectional) │ (None, 100, 256)       │       116,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 100, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ (None, 128)            │       123,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ L1 (Dense)                      │ (None, 252)            │        32,508 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ L2 (Dense)                      │ (None, 128)            │        32,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ L3 (Dense)                      │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ L4 (Dense)                      │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 24)             │           792 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 949,214 (3.62 MB)

 Trainable params: 316,404 (1.21 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 632,810 (2.41 MB)

In [13]:
model = tf.keras.models.load_model("../models/Final_training/third_model_3(0.81455).keras")

In [16]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0000325),
    metrics=['accuracy']
)




reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',  # Choose the metric to monitor
    factor=0.5,          # Factor by which to reduce the learning rate
    patience=3,          # Number of epochs with no improvement before reducing
    min_lr=1e-6          # Minimum learning rate
)

checkpoint = ModelCheckpoint(
    '../models/Final_training/third_model_5.keras',             # File to save the best model
    monitor='val_accuracy',       # Metric to monitor for improvement
    mode='max',                   # Mode 'max' for accuracy (since higher is better)
    save_best_only=True,          # Save only when there is an improvement
    verbose=1,                    # Print message when saving
                    
)


history = model.fit(
    train_dataset,
    validation_data = val_dataset,
    epochs=20,
    callbacks=[reduce_lr, checkpoint]
)

Epoch 1/20
4020/4020 ━━━━━━━━━━━━━━━━━━━━ 0s 234ms/step - accuracy: 0.7762 - loss: 0.6358
Epoch 1: val_accuracy improved from -inf to 0.83685, saving model to ../models/Final_training/third_model_5.keras
4020/4020 ━━━━━━━━━━━━━━━━━━━━ 1013s 251ms/step - accuracy: 0.7762 - loss: 0.6358 - val_accuracy: 0.8368 - val_loss: 0.4740 - learning_rate: 3.2500e-05
Epoch 2/20
4020/4020 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step - accuracy: 0.7753 - loss: 0.6289
Epoch 2: val_accuracy improved from 0.83685 to 0.83706, saving model to ../models/Final_training/third_model_5.keras
4020/4020 ━━━━━━━━━━━━━━━━━━━━ 1030s 256ms/step - accuracy: 0.7753 - loss: 0.6289 - val_accuracy: 0.8371 - val_loss: 0.4728 - learning_rate: 3.2500e-05
Epoch 3/20
4020/4020 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step - accuracy: 0.7791 - loss: 0.6215
Epoch 3: val_accuracy improved from 0.83706 to 0.83803, saving model to ../models/Final_training/third_model_5.keras
4020/4020 ━━━━━━━━━━━━━━━━━━━━ 999s 248ms/step - accuracy: 0.7791 - loss: 0.62